In [1]:
from gears.gears_second_emb import GEARS
from gears import PertData

/Users/rsorenso/Documents/GitHub/GEARS/gears_venv/lib/python3.13/site-packages/anndata/__init__.py:44: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  return module_get_attr_redirect(attr_name, deprecated_mapping=_DEPRECATED)
/Users/rsorenso/Documents/GitHub/GEARS/gears_venv/lib/python3.13/site-packages/anndata/__init__.py:44: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  return module_get_attr_redirect(attr_name, deprecated_mapping=_DEPRECATED)
/Users/rsorenso/Documents/GitHub/GEARS/gears_venv/lib/python3.13/site-packages/anndata/__init__.py:44: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  return module_get_attr_redirect(attr_name, deprecated_mapping=_DEPRECATED)
/Users/rsorenso/Documents/GitHub/GEARS/gears_venv/lib/python3.13/site-packages/anndata/__init__.py:44: FutureWarning: Importing read_loom from `anndata` is 

In [3]:
pert_data = PertData('./data')
pert_data.load(data_name = 'norman')
pert_data.prepare_split(split = 'simulation', seed = 1)
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128)

Found local copy...
Found local copy...
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['RHOXF2BB+ctrl' 'LYL1+IER5L' 'ctrl+IER5L' 'KIAA1804+ctrl' 'IER5L+ctrl'
 'RHOXF2BB+ZBTB25' 'RHOXF2BB+SET']
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:9
combo_seen1:43
combo_seen2:19
unseen_single:36
Done!
Creating dataloaders....
Done!


here1


In [7]:
import pickle
import torch

# Load and convert custom perturbation graph
with open("demo/BioGRID_graph.pkl", "rb") as f:
    G_nx = pickle.load(f)

node_map = pert_data.node_map_pert

def nx_to_gears_graph(G_nx, node_map):
    valid_edges = [(u, v, d.get('weight', 1.0))
                   for u, v, d in G_nx.edges(data=True)
                   if u in node_map and v in node_map]
    # shape = (2, num_edges) each column is an edge
    edge_index = torch.LongTensor([
        [node_map[u] for u, v, _ in valid_edges],
        [node_map[v] for u, v, _ in valid_edges]
    ])
    edge_weight = torch.FloatTensor([w for _, _, w in valid_edges])
    return edge_index, edge_weight

G_BioGRID, G_BioGRID_weight = nx_to_gears_graph(G_nx, node_map)

Create a model object; if you use [wandb](https://wandb.ai), you can easily track model training and evaluation by setting `weight_bias_track` to true, and specify the `proj_name` and `exp_name` that you like.

In [10]:
gears_model = GEARS(pert_data, device = 'cpu', # cuda:7 not available
                        weight_bias_track = False, 
                        proj_name = 'pertnet_1_0_dual', 
                        exp_name = 'pertnet_1_0_dual')
gears_model.model_initialize(
    hidden_size = 64,
    G_BioGRID = G_BioGRID,
    G_BioGRID_weight = G_BioGRID_weight
)

Found local copy...


You can find available tunable parameters in model_initialize via

In [11]:
gears_model.tunable_parameters()

{'hidden_size': 'hidden dimension, default 64',
 'num_go_gnn_layers': 'number of GNN layers for GO graph, default 1',
 'num_gene_gnn_layers': 'number of GNN layers for co-expression gene graph, default 1',
 'decoder_hidden_size': 'hidden dimension for gene-specific decoder, default 16',
 'num_similar_genes_go_graph': 'number of maximum similar K genes in the GO graph, default 20',
 'num_similar_genes_co_express_graph': 'number of maximum similar K genes in the co expression graph, default 20',
 'coexpress_threshold': 'pearson correlation threshold when constructing coexpression graph, default 0.4',
 'uncertainty': 'whether or not to turn on uncertainty mode, default False',
 'uncertainty_reg': 'regularization term to balance uncertainty loss and prediction loss, default 1',
 'direction_lambda': 'regularization term to balance direction loss and prediction loss, default 1'}

Train your model:

Note: For the sake of demo, we set epoch size to 1. To get full model, set `epochs = 20`.

In [12]:
gears_model.train(epochs = 1, lr = 1e-3)

Start Training...
Epoch 1 Step 1 Train Loss: 0.5063
Epoch 1 Step 51 Train Loss: 0.4738
Epoch 1 Step 101 Train Loss: 0.5014
Epoch 1 Step 151 Train Loss: 0.4370
Epoch 1 Step 201 Train Loss: 0.4209
Epoch 1 Step 251 Train Loss: 0.5085
Epoch 1 Step 301 Train Loss: 0.5208
Epoch 1 Step 351 Train Loss: 0.4528
Epoch 1 Step 401 Train Loss: 0.4527
Epoch 1 Step 451 Train Loss: 0.5220
Epoch 1 Step 501 Train Loss: 0.5892
Epoch 1 Step 551 Train Loss: 0.4842
Epoch 1 Step 601 Train Loss: 0.4385
Epoch 1 Step 651 Train Loss: 0.5087
Epoch 1 Step 701 Train Loss: 0.6120
Epoch 1 Step 751 Train Loss: 0.4599
Epoch 1 Step 801 Train Loss: 0.4192
Epoch 1 Step 851 Train Loss: 0.4145
Epoch 1 Step 901 Train Loss: 0.4816
Epoch 1 Step 951 Train Loss: 0.5403


KeyboardInterrupt: 

Save and load pretrained models:

In [ ]:
gears_model.save_model('test_model_1_0_dual')
gears_model.load_pretrained('test_model_1_0_dual')

Make prediction for new perturbation:

In [ ]:
gears_model.predict([['FEV'], ['FEV', 'AHR']])

Gene list can be found here:

In [ ]:
gears_model.gene_list[:5]